In [2]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import easyocr
import cv2
import glob
import matplotlib.pyplot as plt
import os.path
from flask import Flask,request,Response
import json
import requests
import face_recognition
from PIL import Image, ImageDraw
from IPython.display import display
import numpy as np

In [3]:
import pandas as pd
import tensorflow as tf
import pytesseract 
import matplotlib.pyplot as plt
import imutils
import xml.etree.ElementTree as xet
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import re

In [4]:
# Load model
import easyocr
# reader = easyocr.Reader(['en'])

In [5]:
def face_recog():
    folder_path = r'C:/Users/Igloble/WebDownload/LiveSnapshot/IR_7F092EDPAGC71E9/'
    file_type = r'/*jpg'
    def remove_du(dupflo):
        nodo=[]
        for element in dupflo:
            if element not in nodo:
                nodo.append(element)
        return nodo
        print(remove_du(folder_path))
    
# print(folder_path+file_type)
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)
    img_1=os.path.basename(max_file)
    print(img_1)
#Face recognition
    if ".jpg" not in img_1:
        print(img_1)
        return None
        
    print('hello')    
    unknown_image = face_recognition.load_image_file("C:/Users/Igloble/WebDownload/LiveSnapshot/IR_7F092EDPAGC71E9/"+img_1)
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    pil_image = Image.fromarray(unknown_image)
    # Create a Pillow ImageDraw Draw instance to draw with
    draw = ImageDraw.Draw(pil_image)
    api_url = "http://65.1.91.185:8080/v1/internal/detection"
    todo = {"personId": 0,"requestHost": "INTERNAL","gateId": "gate2"}
    headers =  {"Content-Type":"application/json", "token": "test"}
    # Loop through each face found in the unknown image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_encoded_array, face_encoding)
        print(matches)

        id = 0

    # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_encoded_array, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            id = resource_array[best_match_index]

        print(id)
        if id == 0:
            draw.rectangle(((left, top), (right, bottom)), outline=(255, 0, 0))
            # Draw a label with a name below the face
            text_width, text_height = draw.textsize(str(id))
            draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(255, 0, 0), outline=(255, 0, 0))
            draw.text((left + 6, bottom - text_height - 5), str(id), fill=(255, 255, 255, 255))
        else:
            draw.rectangle(((left, top), (right, bottom)), outline=(0, 255, 0))
            # Draw a label with a name below the face
            text_width, text_height = draw.textsize(str(id))
            draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 255, 0), outline=(0, 255, 0))
            draw.text((left + 6, bottom - text_height - 5), str(id), fill=(255, 255, 255, 255))

        todo['personId'] = id
        todo['localImage']=img_1
        print(todo)
        response = requests.post(api_url, data=json.dumps(todo), headers=headers)
        print(response)

    del draw
    # Display the resulting image
    display(pil_image)
    im1 = pil_image.save("C:/Users/Igloble/Desktop/Pract/Project1/so/"+img_1)

def on_created(event):
    print('created')
#     number_plate()
    object_detection()
    face_recog()
    
def on_deleted(event):
    print('deleted')

def on_modified(event):
    print('modified')
    
def on_moved(event):
    print('moved')


if __name__ == "__main__":
    event_handler=FileSystemEventHandler()
    #Calling Function
    event_handler.on_created=on_created
    event_handler.on_deleted=on_deleted
    event_handler.on_modified=on_modified
    event_handler.on_moved=on_moved
    
    
    path="C:/Users/Public/SmartPSS/Capture/"
    
    observer=Observer()
    observer.schedule(event_handler,path,recursive=True)
    observer.start()
    try:
        print("Monitoring")
        api_url = "http://65.1.91.185:8080/v1/internal/encodings"
        headers =  { "token": "test"}
        response = requests.get(api_url,headers=headers)
        data=response.json()
    
        for i in range(0,len(data)):
            obj=data[i]
            known_encoded_array.append(np.array(json.loads(obj['encodedValue'])))
            resource_array.append(obj['resourceId'])
        print(resource_array)
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print('done')
    observer.join()
    
class Handler(FileSystemEventHandler):
    @staticmethod
    def on_any_event(event):
        # if event.is_directory:
        #     return None
        print(
            "[{}] noticed: [{}] on: [{}] ".format(
                time.asctime(), event.event_type, event.src_path
            )
        )

if __name__ == "__main__":
    w = Watcher(".")
    w.run()

Monitoring


ConnectTimeout: HTTPConnectionPool(host='65.1.91.185', port=8080): Max retries exceeded with url: /v1/internal/encodings (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000288B09EF520>, 'Connection to 65.1.91.185 timed out. (connect timeout=None)'))